# 1. 데이터 읽기

In [2]:
import pandas as pd
import numpy as np
import requests, lxml
from bs4 import BeautifulSoup

## 1.1 데이터 불러오기

기상청에서 별도로 제공해준 데이터에 행정구역별 위도와 경도가 나와있다. 이를 이용해서 지역의 2단계 depth(시군구)에 해당하는 좌표를 얻을 수 있다. 그러나 배달데이터는 오류동이지만 날씨데이터에서는 오류1동, 오류2동...등과 같이 세분화 되어있기 때문에 이런 데이터는 **카카오 API** 중 주소검색을 이용하여 위도와 경도 값을 얻는다.

In [3]:
weather = pd.read_excel('/content/drive/MyDrive/기상청41_단기예보 조회서비스_오픈API활용가이드_격자_위경도(20210401).xlsx', engine='openpyxl')
weather

,구분,행정구역코드,1단계,2단계,3단계,격자 X,격자 Y,경도(시),경도(분),경도(초),위도(시),위도(분),위도(초),경도(초/100),위도(초/100),위치업데이트,Unnamed: 16
0,kor,1100000000,서울특별시,NaN,NaN,60,127,126,58,48.03,37,33,48.85,126.980008,37.563569,NaN,NaN
1,kor,1111000000,서울특별시,종로구,NaN,60,127,126,58,53.91,37,34,13.36,126.981642,37.570378,NaN,NaN
2,kor,1111051500,서울특별시,종로구,청운효자동,60,127,126,58,14.35,37,35,2.89,126.970652,37.584137,NaN,NaN
3,kor,1111053000,서울특별시,종로구,사직동,60,127,126,58,15.44,37,34,23.77,126.970956,37.573269,NaN,NaN
4,kor,1111054000,서울특별시,종로구,삼청동,60,127,126,59,2.32,37,34,56.73,126.983978,37.582425,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3769,kor,5013060000,제주특별자치도,서귀포시,대천동,52,32,126,28,46.47,33,14,50.91,126.479575,33.247475,NaN,NaN
3770,kor,5013061000,제주특별자치도,서귀포시,중문동,51,32,126,26,12.40,33,14,53.44,126.436778,33.248178,NaN,NaN
3771,kor,5013062000,제주특별자치도,서귀포시,예래동,50,32,126,23,59.67,33,15,3.69,126.399908,33.251025,NaN,NaN
3772,kor,5019000000,이어도,NaN,NaN,28,8,0,0,0.00,0,0,0.00,0.000000,0.000000,NaN,NaN


In [4]:
#필요없는 열 제거
weather = weather.drop(['구분', '위치업데이트', 'Unnamed: 16'], axis=1)
#이어도 제거
weather = weather[weather['1단계']!='이어도']
weather

,행정구역코드,1단계,2단계,3단계,격자 X,격자 Y,경도(시),경도(분),경도(초),위도(시),위도(분),위도(초),경도(초/100),위도(초/100)
0,1100000000,서울특별시,NaN,NaN,60,127,126,58,48.03,37,33,48.85,126.980008,37.563569
1,1111000000,서울특별시,종로구,NaN,60,127,126,58,53.91,37,34,13.36,126.981642,37.570378
2,1111051500,서울특별시,종로구,청운효자동,60,127,126,58,14.35,37,35,2.89,126.970652,37.584137
3,1111053000,서울특별시,종로구,사직동,60,127,126,58,15.44,37,34,23.77,126.970956,37.573269
4,1111054000,서울특별시,종로구,삼청동,60,127,126,59,2.32,37,34,56.73,126.983978,37.582425
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3767,5013058000,제주특별자치도,서귀포시,서홍동,53,33,126,33,9.84,33,15,38.28,126.552733,33.260633
3768,5013059000,제주특별자치도,서귀포시,대륜동,52,32,126,30,48.28,33,14,41.22,126.513411,33.244783
3769,5013060000,제주특별자치도,서귀포시,대천동,52,32,126,28,46.47,33,14,50.91,126.479575,33.247475
3770,5013061000,제주특별자치도,서귀포시,중문동,51,32,126,26,12.40,33,14,53.44,126.436778,33.248178


In [5]:
weather.describe()

,행정구역코드,격자 X,격자 Y,경도(시),경도(분),경도(초),위도(시),위도(분),위도(초),경도(초/100),위도(초/100)
count,3.772000e+03,3772.000000,3772.000000,3772.000000,3772.00000,3772.000000,3772.000000,3772.000000,3772.000000,3772.000000,3772.000000
mean,3.732887e+09,71.003181,101.151379,122.605514,28.10684,29.488337,34.621686,28.390509,29.143044,127.595226,36.378593
std,1.189482e+09,15.539259,23.235336,23.462248,19.60904,17.818969,6.704490,16.646510,17.905719,0.876783,1.072544
min,1.100000e+09,21.000000,32.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,124.714100,33.223572
25%,2.824572e+09,59.000000,80.000000,126.000000,8.00000,14.150000,35.000000,14.000000,13.682500,126.917411,35.388358
50%,4.213028e+09,64.000000,101.000000,127.000000,29.00000,30.015000,36.000000,29.000000,29.240000,127.219115,36.389263
75%,4.623052e+09,85.000000,124.000000,128.000000,46.00000,44.880000,37.000000,41.000000,44.857500,128.404465,37.452550
max,5.013062e+09,144.000000,147.000000,130.000000,59.00000,65.210000,38.000000,59.000000,59.981880,131.864847,38.491644


In [6]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3772 entries, 0 to 3771
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   행정구역코드     3772 non-null   int64  
 1   1단계        3772 non-null   object 
 2   2단계        3755 non-null   object 
 3   3단계        3505 non-null   object 
 4   격자 X       3772 non-null   int64  
 5   격자 Y       3772 non-null   int64  
 6   경도(시)      3772 non-null   int64  
 7   경도(분)      3772 non-null   int64  
 8   경도(초)      3772 non-null   float64
 9   위도(시)      3772 non-null   int64  
 10  위도(분)      3772 non-null   int64  
 11  위도(초)      3772 non-null   float64
 12  경도(초/100)  3772 non-null   float64
 13  위도(초/100)  3772 non-null   float64
dtypes: float64(4), int64(7), object(3)
memory usage: 442.0+ KB


In [7]:
weather.isna().sum()

행정구역코드         0
1단계            0
2단계           17
3단계          267
격자 X           0
격자 Y           0
경도(시)          0
경도(분)          0
경도(초)          0
위도(시)          0
위도(분)          0
위도(초)          0
경도(초/100)      0
위도(초/100)      0
dtype: int64

In [8]:
!unzip /content/drive/MyDrive/elicer_col_name.zip -d /content/elice_배달데이터

df_2019=pd.read_csv("/content/elice_배달데이터/elicer_2019_col_name.csv", parse_dates=['날짜'])
df_2020_1=pd.read_csv("/content/elice_배달데이터/elicer_2020_1_col_name.csv", parse_dates=['날짜'])
df_2020_2=pd.read_csv("/content/elice_배달데이터/elicer_2020_2_col_name.csv", parse_dates=['날짜'])
df_2021=pd.read_csv("/content/elice_배달데이터/elicer_2021_col_name.csv", parse_dates=['날짜'])

df = pd.concat([df_2019, df_2020_1, df_2020_2, df_2021], ignore_index=True)
df

Archive:  /content/drive/MyDrive/elicer_col_name.zip
  inflating: /content/elice_배달데이터/elicer_2019_col_name.csv  
  inflating: /content/elice_배달데이터/elicer_2020_1_col_name.csv  
  inflating: /content/elice_배달데이터/elicer_2020_2_col_name.csv  
  inflating: /content/elice_배달데이터/elicer_2021_col_name.csv  


,날짜,시간대,광역시도,시군구,읍면동,배달건수
0,2019-08-01,0,강원도,속초시,교동,1
1,2019-08-01,0,경기도,고양시 일산동구,마두동,4
2,2019-08-01,0,경기도,고양시 일산동구,백석동,28
3,2019-08-01,0,경기도,고양시 일산동구,식사동,3
4,2019-08-01,0,경기도,고양시 일산동구,장항동,4
...,...,...,...,...,...,...
2382976,2021-08-31,23,충청남도,천안시 서북구,쌍용동,6
2382977,2021-08-31,23,충청북도,제천시,신백동,6
2382978,2021-08-31,23,충청북도,제천시,신월동,2
2382979,2021-08-31,23,충청북도,제천시,천남동,1


## 1.2 데이터 존재 여부 확인하기

level1과 level2는 배달데이터의 지역을 모두 포함한다.

In [9]:
level2 = list(weather['2단계'].unique())
SiGunGu = list(df['시군구'].unique())

yes =[]
no=[]

for region in SiGunGu:
  #배달데이터는 시군구 데이터에 띄어쓰기가 되어있는 것이 있는데 비교를 위해 공백을 제거해 날씨데이터와 동일한 모양으로 만든다.
  #왜인지는 모르겠으나,,,창원시는 예외이다. 창원시는 띄어쓰기가 되어있다.
  if not region.startswith('창원') :
    region = region.replace(" ","") 
  
  if region in level2:
    yes.append(region)
  else:
    no.append(region)

print(no) #모두 존재! (\\N은 세종특별자치시이다. 배달데이터는 세종특별자치시에 값이 들어있지 않다.)

['\\N']


In [10]:
df[df['시군구']=='\\N']

,날짜,시간대,광역시도,시군구,읍면동,배달건수
15861,2019-08-08,12,세종특별자치시,\N,소담동,1
16213,2019-08-08,14,세종특별자치시,\N,소담동,4
16403,2019-08-08,15,세종특별자치시,\N,소담동,3
16588,2019-08-08,16,세종특별자치시,\N,소담동,3
16790,2019-08-08,17,세종특별자치시,\N,소담동,7
...,...,...,...,...,...,...
1941128,2021-03-13,23,세종특별자치시,\N,조치원읍,6
1941262,2021-03-14,0,세종특별자치시,\N,조치원읍,9
1941355,2021-03-14,1,세종특별자치시,\N,조치원읍,3
1942191,2021-03-14,12,세종특별자치시,\N,조치원읍,1


no list에 들어있는 값들은 카카오 API 주소검색을 이용해 위도 경도 값을 얻는다.

In [11]:
level3 = list(weather['3단계'].unique())
Dong = list(df['읍면동'].unique())

yes =[]
no=[]

for region in Dong:
  #배달데이터는 시군구 데이터에 띄어쓰기가 되어있는 것이 있는데 비교를 위해 공백을 제거해 날씨데이터와 동일한 모양으로 만든다.
  #왜인지는 모르겠으나,,,창원시는 예외이다. 창원시는 띄어쓰기가 되어있다.
  if not region.startswith('창원') :
    region = region.replace(" ","") 
  
  if region in level3:
    yes.append(region)
  else:
    no.append(region)

print(len(no))
print(no)

444
['마두동', '장항동', '광명동', '철산동', '하안동', '조남동', '금오동', '신곡동', '용현동', '의정부동', '호원동', '삼방동', '삼정동', '계동', '본성동', '봉곡동', '개봉동', '고척동', '구로동', '독산동', '망원동', '대림동', '조례동', '효자동2가', '효자동3가', '서귀동', '석림동', '의림동', '장락동', '소하동', '중안동', '어방동', '외동', '인사동', '본오동', '봉황동', '삼계동', '장대동', '칠암동', '양덕동', '합성동', '정하동', '화곡동', '궁동', '방학동', '정릉동', '풍동', '덕이동', '주엽동', '일직동', '감정동', '괴안동', '옥길동', '남사면', '동패동', '신읍동', '산척동', '대안동', '옥봉동', '옥야동', '천리동', '이곡동', '명지동', '하단동', '도봉동', '창동', '신정동', '영등포동1가', '연향동', '왕지동', '명천동', '강제동', '모산동', '중앙로1가', '중앙로2가', '태장동', '산현동', '서상동', '소태동', '가양동', '상계동', '쌍문동', '신대방동', '청천동', '하소동', '낙양동', '내당동', '평리동', '신호동', '방화동', '산곡동', '청당동', '반곡동', '오산동', '오치동', '파호동', '등촌동', '마곡동', '온수동', '죽정동', '고암동', '논곡동', '와동동', '내발산동', '신월동', '십정동', '야당동', '벌리동', '수정동', '길음동', '주약동', '신포동1가', '광석동', '정상동', '천왕동', '홍은동', '화봉동', '예천동', '가좌동', '망경동', '다대동', '율세동', '불광동', '천남동', '삼도이동', '목동동', '간석동', '동대동', '장림동', '지곡동', '초전동', '인의동', '나운동', '관설동', '동현동', '아라일동', '회원동', '봉천동', '광안동', '우동', '범박

In [12]:
print(len(yes)) #382개는 weather데이터에서 얻을 수 있다.

382


# 2. 위도 경도 데이터 프레임 만들기

## 2.1 level1 (광역시도)

In [13]:
df_level1= pd.DataFrame(columns=['광역시도','시군구','읍면동','경도(초/100)','위도(초/100)'])
df_level1

,광역시도,시군구,읍면동,경도(초/100),위도(초/100)


In [14]:
print(len(weather.loc[(weather['1단계']=='세종특별자치시') & (weather['2단계'].isna()) & (weather['3단계'].isna()),'경도(초/100)']))
weather.loc[(weather['1단계']=='세종특별자치시') & (weather['2단계'].isna()) & (weather['3단계'].isna()),'경도(초/100)'].values[0]

1


127.28906909999999

In [15]:
level1_list = list(df['광역시도'].unique())
i=0
for Si_Do in level1_list:
  x = weather[(weather['1단계']==Si_Do) & (weather['2단계'].isna()) & (weather['3단계'].isna())]['경도(초/100)'].values[0]
  y = weather[(weather['1단계']==Si_Do) & (weather['2단계'].isna()) & (weather['3단계'].isna())]['위도(초/100)'].values[0]
  df_level1.loc[i] = [Si_Do, np.NaN, np.NaN, x, y]
  i+=1

df_level1

,광역시도,시군구,읍면동,경도(초/100),위도(초/100)
0,강원도,NaN,NaN,127.731975,37.882692
1,경기도,NaN,NaN,127.011689,37.271844
2,경상남도,NaN,NaN,128.694167,35.234736
3,대전광역시,NaN,NaN,127.386567,36.347119
4,서울특별시,NaN,NaN,126.980008,37.563569
5,전라남도,NaN,NaN,126.465000,34.813044
6,전라북도,NaN,NaN,127.111053,35.817275
7,제주특별자치도,NaN,NaN,126.500333,33.485694
8,충청남도,NaN,NaN,127.422956,36.323872
9,충청북도,NaN,NaN,127.493586,36.632500


In [16]:
df_level1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17 entries, 0 to 16
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   광역시도       17 non-null     object 
 1   시군구        0 non-null      float64
 2   읍면동        0 non-null      float64
 3   경도(초/100)  17 non-null     float64
 4   위도(초/100)  17 non-null     float64
dtypes: float64(4), object(1)
memory usage: 816.0+ bytes


In [17]:
# df_level1.to_csv("/content/drive/MyDrive/경도_위도_level1.csv", index=False, encoding="utf-8-sig")

## 2.2 level2 (시군구)

In [18]:
df_level2= pd.DataFrame(columns=['광역시도','시군구','읍면동','경도(초/100)','위도(초/100)'])
df_level2

,광역시도,시군구,읍면동,경도(초/100),위도(초/100)


In [19]:
#세종특별시의 경우 날씨데이터는 배달데이터와 다르게 시군구 지역도 세종특별자치시로 채워져있음
weather[weather['1단계']=='세종특별자치시']

,행정구역코드,1단계,2단계,3단계,격자 X,격자 Y,경도(시),경도(분),경도(초),위도(시),위도(분),위도(초),경도(초/100),위도(초/100)
1239,3600000000,세종특별자치시,NaN,NaN,66,103,0,0,0.00000,0,0,0.00000,127.289069,36.480012
1240,3611000000,세종특별자치시,세종특별자치시,NaN,66,103,0,0,0.00000,0,0,0.00000,127.289069,36.480012
1241,3611025000,세종특별자치시,세종특별자치시,조치원읍,66,106,0,0,0.00000,0,0,0.00000,127.298399,36.604528
1242,3611031000,세종특별자치시,세종특별자치시,연기면,65,105,0,0,0.00000,0,0,0.00000,127.273774,36.541874
1243,3611032000,세종특별자치시,세종특별자치시,연동면,66,105,0,0,0.00000,0,0,0.00000,127.326866,36.559089
1244,3611033000,세종특별자치시,세종특별자치시,부강면,67,104,0,0,0.00000,0,0,0.00000,127.370376,36.527112
1245,3611034000,세종특별자치시,세종특별자치시,금남면,65,103,0,0,0.00000,0,0,0.00000,127.280350,36.463826
1246,3611035000,세종특별자치시,세종특별자치시,장군면,64,104,0,0,0.00000,0,0,0.00000,127.205601,36.496793
1247,3611036000,세종특별자치시,세종특별자치시,연서면,65,106,0,0,0.00000,0,0,0.00000,127.271622,36.592587
1248,3611037000,세종특별자치시,세종특별자치시,전의면,64,108,0,0,0.00000,0,0,0.00000,127.195513,36.681251


**강서구**의 경우 부산광역시 강서구도 있고, 서울특별시 강서구도 존재한다. 이름이 같은 지역이 있을 수 있으므로 `df['시군구'].unique()` 만큼만 도는 것은 매우 위험. 그리고 데이터를 넣을 때 1단계도 동시에 채워넣어야 하므로 **광역시도별 시군구의 unique()로 적용**하자!


In [20]:
df[df['시군구']=='강서구']

,날짜,시간대,광역시도,시군구,읍면동,배달건수
175,2019-08-01,10,서울특별시,강서구,염창동,1
176,2019-08-01,10,서울특별시,강서구,화곡동,2
265,2019-08-01,11,부산광역시,강서구,명지동,22
268,2019-08-01,11,서울특별시,강서구,공항동,1
269,2019-08-01,11,서울특별시,강서구,화곡동,5
...,...,...,...,...,...,...
2382684,2021-08-31,21,서울특별시,강서구,화곡동,4
2382835,2021-08-31,22,부산광역시,강서구,명지동,31
2382838,2021-08-31,22,서울특별시,강서구,화곡동,3
2382945,2021-08-31,23,부산광역시,강서구,명지동,8


In [21]:
i=0
level1_list = list(df['광역시도'].unique())

for Si_Do in level1_list:
  level2_list = list(df[df['광역시도']==Si_Do]['시군구'].unique())
  for SiGunGu in level2_list:
  #배달데이터는 시군구 데이터에 띄어쓰기가 되어있는 것이 있는데 비교를 위해 공백을 제거해 날씨데이터와 동일한 모양으로 만든다.
  #왜인지는 모르겠으나,,,창원시는 예외이다. 창원시는 띄어쓰기가 되어있다.
    if not SiGunGu.startswith('창원') :
      SiGunGu = SiGunGu.replace(" ","")
    if SiGunGu == '\\N': #세종특별자치시
      SiGunGu = '세종특별자치시'

    # 날씨데이터에 원하는 데이터가(배달데이터의 시군구) 있는지 다시한번 확인. 없거나 많으면 해당 시군구와 값을 출력한다.
    # if len(weather.loc[(weather['1단계']==Si_Do) & (weather['2단계']==SiGunGu) & (weather['3단계'].isna()),'경도(초/100)']) != 1:
    #   print(SiGunGu, weather.loc[(weather['1단계']==Si_Do) & (weather['2단계']==SiGunGu) & (weather['3단계'].isna()),'경도(초/100)'])
    # if len(weather.loc[(weather['1단계']==Si_Do) & (weather['2단계']==SiGunGu) & (weather['3단계'].isna()),'위도(초/100)']) != 1:
    #   print(SiGunGu, weather.loc[(weather['1단계']==Si_Do) & (weather['2단계']==SiGunGu) & (weather['3단계'].isna()),'위도(초/100)'])
      
    x=weather.loc[(weather['1단계']==Si_Do) & (weather['2단계']==SiGunGu) & (weather['3단계'].isna()),'경도(초/100)'].values[0]
    y=weather.loc[(weather['1단계']==Si_Do) & (weather['2단계']==SiGunGu) & (weather['3단계'].isna()),'위도(초/100)'].values[0]
    df_level2.loc[i] = [Si_Do, SiGunGu, np.NaN, x, y]
    i +=1

df_level2

,광역시도,시군구,읍면동,경도(초/100),위도(초/100)
0,강원도,속초시,NaN,128.594167,38.204275
1,강원도,원주시,NaN,127.922056,37.339083
2,강원도,삼척시,NaN,129.167489,37.447086
3,강원도,강릉시,NaN,128.878497,37.749136
4,강원도,철원군,NaN,127.315733,38.144056
...,...,...,...,...,...
150,인천광역시,계양구,NaN,126.739753,37.534792
151,울산광역시,북구,NaN,129.363544,35.579689
152,울산광역시,울주군,NaN,129.297164,35.530739
153,울산광역시,중구,NaN,129.334875,35.566342


df['시군구'].unique()는 141개 뿐이지만 모든 광역시도의 unique()를 적용한 결과 총 **155**개의 데이터를 얻었다. 10개이상 겹치는 이름이 있었다. 

In [22]:
total3=0
total2=0
level1_list = list(df['광역시도'].unique())

for Si_Do in level1_list:
  level2_list = list(df[df['광역시도']==Si_Do]['시군구'].unique())
  total2 += len(level2_list)
  for SiGunGu in level2_list:
    total3 += len(list(df[(df['광역시도']==Si_Do) & (df['시군구']==SiGunGu)]['읍면동'].unique()))

total2, total3

(155, 913)

In [23]:
df_level2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 155 entries, 0 to 154
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   광역시도       155 non-null    object 
 1   시군구        155 non-null    object 
 2   읍면동        0 non-null      float64
 3   경도(초/100)  155 non-null    float64
 4   위도(초/100)  155 non-null    float64
dtypes: float64(3), object(2)
memory usage: 7.3+ KB


In [24]:
df_level2.groupby('광역시도').describe()

읍면동               ...  위도(초/100)                                 
        count mean std min  ...        25%        50%        75%        max
광역시도                        ...                                            
강원도       0.0  NaN NaN NaN  ...  37.522599  37.813839  38.077677  38.204275
경기도       0.0  NaN NaN NaN  ...  37.299142  37.402103  37.627998  38.093364
경상남도      0.0  NaN NaN NaN  ...  35.186954  35.220932  35.292966  35.541536
경상북도      0.0  NaN NaN NaN  ...  35.992547  36.136897  36.583631  36.890261
광주광역시     0.0  NaN NaN NaN  ...  35.136428  35.143100  35.149483  35.171125
대구광역시     0.0  NaN NaN NaN  ...  35.838965  35.860747  35.872271  35.883617
대전광역시     0.0  NaN NaN NaN  ...  36.324133  36.343794  36.352483  36.359367
부산광역시     0.0  NaN NaN NaN  ...  35.142467  35.173186  35.201872  35.241350
서울특별시     0.0  NaN NaN NaN  ...  37.512993  37.548156  37.581375  37.665833
세종특별자치시   0.0  NaN NaN NaN  ...  36.480012  36.480012  36.480012  36.480012
울산광역시     0.0  NaN NaN NaN  ...  35.548540  35.566342  35.573015  35.579689
인천광역시     0.0  NaN NaN NaN  ...  37.448560  37.482474  37.527160  37.542692
전라남도      0.0  NaN NaN NaN  ...  34.808789  34.937536  34.947606  35.012839
전라북도      0.0  NaN NaN NaN  ...  35.800575  35.809189  35.945275  35.964642
제주특별자치도   0.0  NaN NaN NaN  ...  33.313340  33.374331  33.435321  33.496311
충청남도      0.0  NaN NaN NaN  ...  36.464468  36.782097  36.795615  36.875558
충청북도      0.0  NaN NaN NaN  ...  36.747712  36.874918  36.983381  37.129769

[17 rows x 24 columns]

In [25]:
# df_level2.to_csv("/content/drive/MyDrive/경도_위도_level2.csv", index=False, encoding="utf-8-sig")

## 2.3 level3 (읍면동)

다음처럼 **groupby를 연속**해서 사용하면 더 간단하게 읍면동 개수를 얻을 수 있다! for문을 중첩해서 unique()를 사용해 리스트를 만드는 것보다 훨씬 빠르게 계산된다. </br>
최종 데이터는 **913**개여야 한다.

In [31]:
df.groupby(['광역시도','시군구','읍면동']).nunique().reset_index()

,광역시도,시군구,읍면동,날짜,시간대,배달건수
0,강원도,강릉시,교동,485,13,9
1,강원도,강릉시,송정동,24,9,3
2,강원도,삼척시,교동,50,12,4
3,강원도,삼척시,남양동,699,13,7
4,강원도,속초시,교동,117,15,26
...,...,...,...,...,...,...
908,충청북도,충주시,연수동,70,16,33
909,충청북도,충주시,용산동,89,15,20
910,충청북도,충주시,중앙탑면,1,5,3
911,충청북도,충주시,칠금동,4,15,18


### 2.3.1 날씨데이터에서 채우기

In [32]:
df_level3= pd.DataFrame(columns=['광역시도','시군구','읍면동','경도(초/100)','위도(초/100)'])
df_level3

,광역시도,시군구,읍면동,경도(초/100),위도(초/100)


In [33]:
weather[weather['2단계']=='세종특별자치시']
weather.loc[(weather['1단계']=='세종특별자치시') & (weather['2단계']=='세종특별자치시') & (weather['3단계']=='조치원읍'),'경도(초/100)'].values[0]

127.298399

In [38]:
level3_list=df.groupby(['광역시도','시군구','읍면동']).nunique().reset_index()['읍면동'].values

In [39]:
level3_list

array(['교동', '송정동', '교동', '남양동', '교동', '금호동', '조양동', '중앙동', '청학동', '관설동',
       '단계동', '단구동', '무실동', '반곡동', '일산동', '지정면', '태장동', '동송읍', '서면',
       '동내면', '석사동', '소양로2가', '요선동', '퇴계동', '효자동', '후평동', '도내동', '동산동',
       '삼송동', '성사동', '신원동', '용두동', '원흥동', '주교동', '토당동', '행신동', '화정동',
       '마두동', '백석동', '성석동', '식사동', '장항동', '정발산동', '중산동', '풍동', '대화동',
       '덕이동', '일산동', '주엽동', '탄현동', '광명동', '노온사동', '소하동', '일직동', '철산동',
       '하안동', '경안동', '목동', '송정동', '역동', '오포읍', '장지동', '중대동', '탄벌동', '태전동',
       '갈매동', '도마교동', '부곡동', '감정동', '고촌읍', '구래동', '사우동', '양촌읍', '풍무동',
       '다산동', '별내동', '오남읍', '와부읍', '진건읍', '진접읍', '퇴계원읍', '평내동', '호평동',
       '동두천동', '보산동', '상패동', '생연동', '송내동', '안흥동', '지행동', '고강동', '괴안동',
       '내동', '도당동', '범박동', '삼정동', '상동', '소사동', '소사본동', '송내동', '심곡동',
       '심곡본동', '약대동', '역곡동', '오정동', '옥길동', '원미동', '원종동', '중동', '춘의동',
       '야탑동', '복정동', '수진동', '신흥동', '양지동', '태평동', '금광동', '도촌동', '상대원동',
       '성남동', '여수동', '은행동', '중앙동', '하대원동', '구운동', '금곡동', '호매실동', '천천동',
    

In [40]:
i=0
level1_list = list(df['광역시도'].unique())
no_list=[]

for Si_Do in level1_list:
  level2_list = list(df[df['광역시도']==Si_Do]['시군구'].unique())
  for SiGunGu in level2_list:
    level3_list = list(df[(df['광역시도']==Si_Do) & (df['시군구']==SiGunGu)]['읍면동'].unique()) #시군구를 날씨데이터 포맷으로 바꾸기 전 level3_list를 만들어야한다!
  
    #배달데이터는 시군구 데이터에 띄어쓰기가 되어있는 것이 있는데 비교를 위해 공백을 제거해 날씨데이터와 동일한 모양으로 만든다.
    #왜인지는 모르겠으나,,,창원시는 예외이다. 창원시는 띄어쓰기가 되어있다.
    if not SiGunGu.startswith('창원') :
      SiGunGu = SiGunGu.replace(" ","")
    if SiGunGu == '\\N': #세종특별자치시
      SiGunGu = '세종특별자치시'

    for Dong in level3_list:
      # 날씨데이터에 원하는 데이터가(배달데이터의 시군구) 있는지 확인. 경도와 위도에는 결측치가 없으므로 둘 중 하나라도 없으면 원하는 지역이 존재하지 않는다고 판단.
      if len(weather.loc[(weather['1단계']==Si_Do) & (weather['2단계']==SiGunGu) & (weather['3단계']==Dong),'경도(초/100)']) != 1:
        no_list.append((Si_Do, SiGunGu, Dong)) #no_list에 있는 지역들은 카카오 API를 이용해서 찾을 예정
      else:
        x=weather.loc[(weather['1단계']==Si_Do) & (weather['2단계']==SiGunGu) & (weather['3단계']==Dong),'경도(초/100)'].values[0]
        y=weather.loc[(weather['1단계']==Si_Do) & (weather['2단계']==SiGunGu) & (weather['3단계']==Dong),'위도(초/100)'].values[0]
        df_level3.loc[i] = [Si_Do, SiGunGu, Dong, x, y]
        i +=1

df_level3

,광역시도,시군구,읍면동,경도(초/100),위도(초/100)
0,강원도,속초시,교동,128.584819,38.199786
1,강원도,속초시,금호동,128.590442,38.204186
2,강원도,속초시,조양동,128.593200,38.184881
3,강원도,원주시,단구동,127.962531,37.323353
4,강원도,원주시,일산동,127.944331,37.346825
...,...,...,...,...,...
313,울산광역시,울주군,온양읍,129.283056,35.415903
314,세종특별자치시,세종특별자치시,소담동,127.301191,36.483522
315,세종특별자치시,세종특별자치시,다정동,127.247913,36.493977
316,세종특별자치시,세종특별자치시,고운동,127.236841,36.519776


In [41]:
level1_list = list(df_level3['광역시도'].unique())
yes_df3=[]

for Si_Do in level1_list:
  level2_list = list(df_level3[df_level3['광역시도']==Si_Do]['시군구'].unique())
  for SiGunGu in level2_list:
    level3_list = list(df_level3[(df_level3['광역시도']==Si_Do) & (df_level3['시군구']==SiGunGu)]['읍면동'].unique()) #시군구를 날씨데이터 포맷으로 바꾸기 전 level3_list를 만들어야한다!
    for Dong in level3_list:
      yes_df3.append((Si_Do,SiGunGu,Dong))

print(len(yes_df3))
yes_df3[-20:]

318


[('대구광역시', '북구', '동천동'),
 ('부산광역시', '강서구', '녹산동'),
 ('부산광역시', '사하구', '당리동'),
 ('부산광역시', '기장군', '정관읍'),
 ('부산광역시', '동래구', '명륜동'),
 ('부산광역시', '남구', '용당동'),
 ('광주광역시', '동구', '학동'),
 ('광주광역시', '북구', '용봉동'),
 ('광주광역시', '북구', '신용동'),
 ('광주광역시', '북구', '양산동'),
 ('광주광역시', '서구', '치평동'),
 ('인천광역시', '부평구', '일신동'),
 ('울산광역시', '울주군', '언양읍'),
 ('울산광역시', '울주군', '삼남읍'),
 ('울산광역시', '울주군', '온산읍'),
 ('울산광역시', '울주군', '온양읍'),
 ('세종특별자치시', '세종특별자치시', '소담동'),
 ('세종특별자치시', '세종특별자치시', '다정동'),
 ('세종특별자치시', '세종특별자치시', '고운동'),
 ('세종특별자치시', '세종특별자치시', '조치원읍')]

In [42]:
print(len(no_list))
no_list[:20]

595


[('강원도', '속초시', '청학동'),
 ('강원도', '속초시', '중앙동'),
 ('강원도', '원주시', '태장동'),
 ('강원도', '원주시', '반곡동'),
 ('강원도', '원주시', '관설동'),
 ('강원도', '강릉시', '교동'),
 ('강원도', '춘천시', '후평동'),
 ('강원도', '춘천시', '효자동'),
 ('강원도', '춘천시', '소양로2가'),
 ('강원도', '춘천시', '요선동'),
 ('경기도', '고양시일산동구', '마두동'),
 ('경기도', '고양시일산동구', '백석동'),
 ('경기도', '고양시일산동구', '장항동'),
 ('경기도', '고양시일산동구', '풍동'),
 ('경기도', '고양시일산동구', '성석동'),
 ('경기도', '고양시일산서구', '일산동'),
 ('경기도', '고양시일산서구', '덕이동'),
 ('경기도', '고양시일산서구', '주엽동'),
 ('경기도', '광명시', '광명동'),
 ('경기도', '광명시', '철산동')]

In [43]:
total_list = yes_df3 + no_list
len(total_list)

913

In [44]:
df_level3[df_level3['광역시도']=='세종특별자치시']

,광역시도,시군구,읍면동,경도(초/100),위도(초/100)
314,세종특별자치시,세종특별자치시,소담동,127.301191,36.483522
315,세종특별자치시,세종특별자치시,다정동,127.247913,36.493977
316,세종특별자치시,세종특별자치시,고운동,127.236841,36.519776
317,세종특별자치시,세종특별자치시,조치원읍,127.298399,36.604528


* 배달데이터 : 효자동2가
* 날씨데이터 : 효자2동
* 카카오 API에 검색할 때 : 상관없음

In [45]:
df[df['읍면동']=='효자동2가']

,날짜,시간대,광역시도,시군구,읍면동,배달건수
42,2019-08-01,0,전라북도,전주시 완산구,효자동2가,5
83,2019-08-01,1,전라북도,전주시 완산구,효자동2가,5
292,2019-08-01,11,전라북도,전주시 완산구,효자동2가,2
420,2019-08-01,12,전라북도,전주시 완산구,효자동2가,6
541,2019-08-01,13,전라북도,전주시 완산구,효자동2가,4
...,...,...,...,...,...,...
190002,2019-10-13,19,전라북도,전주시 완산구,효자동2가,6
191053,2019-10-14,10,전라북도,전주시 완산구,효자동2가,1
191264,2019-10-14,11,전라북도,전주시 완산구,효자동2가,1
191500,2019-10-14,12,전라북도,전주시 완산구,효자동2가,4


In [46]:
weather[weather['2단계']=='전주시완산구']

,행정구역코드,1단계,2단계,3단계,격자 X,격자 Y,경도(시),경도(분),경도(초),위도(시),위도(분),위도(초),경도(초/100),위도(초/100)
2463,4511100000,전라북도,전주시완산구,NaN,63,89,127,7,18.9100,35,48,33.0800,127.121919,35.809189
2464,4511151000,전라북도,전주시완산구,중앙동,63,89,127,8,33.7600,35,49,13.9400,127.142711,35.820539
2465,4511153000,전라북도,전주시완산구,풍남동,63,89,127,9,0.6700,35,48,50.5000,127.150186,35.814028
2466,4511160500,전라북도,전주시완산구,노송동,63,89,127,9,37.5900,35,49,17.1400,127.160442,35.821428
2467,4511163500,전라북도,전주시완산구,완산동,63,88,127,8,13.0000,35,48,22.2800,127.136944,35.806189
2468,4511165000,전라북도,전주시완산구,동서학동,63,89,127,9,15.5900,35,48,23.4700,127.154331,35.806519
2469,4511166000,전라북도,전주시완산구,서서학동,63,88,127,9,2.5500,35,48,10.0000,127.150708,35.802778
2470,4511167100,전라북도,전주시완산구,중화산1동,63,89,127,7,36.8800,35,48,41.8000,127.126911,35.811611
2471,4511167200,전라북도,전주시완산구,중화산2동,63,89,127,7,6.3200,35,49,7.1300,127.118422,35.818647
2472,4511168000,전라북도,전주시완산구,서신동,63,89,127,7,3.2800,35,49,40.2500,127.117578,35.827847


### 2.3.2 open API 이용하기

> 카카오 주소 검색 API </br>
https://developers.kakao.com/docs/latest/ko/local/dev-guide#address-coord

In [47]:
df_level3

,광역시도,시군구,읍면동,경도(초/100),위도(초/100)
0,강원도,속초시,교동,128.584819,38.199786
1,강원도,속초시,금호동,128.590442,38.204186
2,강원도,속초시,조양동,128.593200,38.184881
3,강원도,원주시,단구동,127.962531,37.323353
4,강원도,원주시,일산동,127.944331,37.346825
...,...,...,...,...,...
313,울산광역시,울주군,온양읍,129.283056,35.415903
314,세종특별자치시,세종특별자치시,소담동,127.301191,36.483522
315,세종특별자치시,세종특별자치시,다정동,127.247913,36.493977
316,세종특별자치시,세종특별자치시,고운동,127.236841,36.519776


In [48]:
print(len(no_list))
no_list[:20]

595


[('강원도', '속초시', '청학동'),
 ('강원도', '속초시', '중앙동'),
 ('강원도', '원주시', '태장동'),
 ('강원도', '원주시', '반곡동'),
 ('강원도', '원주시', '관설동'),
 ('강원도', '강릉시', '교동'),
 ('강원도', '춘천시', '후평동'),
 ('강원도', '춘천시', '효자동'),
 ('강원도', '춘천시', '소양로2가'),
 ('강원도', '춘천시', '요선동'),
 ('경기도', '고양시일산동구', '마두동'),
 ('경기도', '고양시일산동구', '백석동'),
 ('경기도', '고양시일산동구', '장항동'),
 ('경기도', '고양시일산동구', '풍동'),
 ('경기도', '고양시일산동구', '성석동'),
 ('경기도', '고양시일산서구', '일산동'),
 ('경기도', '고양시일산서구', '덕이동'),
 ('경기도', '고양시일산서구', '주엽동'),
 ('경기도', '광명시', '광명동'),
 ('경기도', '광명시', '철산동')]

In [49]:
url = 'https://dapi.kakao.com/v2/local/search/address.xml' #address.{format} 요청시 format을 지정하지 않으면 json이 기본이다. 응답은 xml과 json형태를 제공한다.
kakao_header = {'Authorization' : 'KakaoAK 215e6c855eb1a7c49e991aab5a007f8a' }
params ={'query' : ''}
i = 318 # df_level3의 마지막 index는 317

In [50]:
for SiDo, SiGunGu, Dong in no_list:
  params['query'] = SiDo +  " " + Dong #쿼리를 바꿔가며 요청을 보낸다.
  response = requests.get(url, headers=kakao_header, params=params)
  soup = BeautifulSoup(response.text, "lxml")

  df_level3.loc[i] = [SiDo, SiGunGu, Dong, soup.find("x").get_text(), soup.find("y").get_text()]
  i +=1

df_level3

,광역시도,시군구,읍면동,경도(초/100),위도(초/100)
0,강원도,속초시,교동,128.585,38.1998
1,강원도,속초시,금호동,128.59,38.2042
2,강원도,속초시,조양동,128.593,38.1849
3,강원도,원주시,단구동,127.963,37.3234
4,강원도,원주시,일산동,127.944,37.3468
...,...,...,...,...,...
908,울산광역시,북구,중산동,129.350843351972,35.6637974175003
909,울산광역시,북구,매곡동,129.367253622611,35.6440061479904
910,울산광역시,북구,천곡동,129.32694539328,35.6469004913624
911,울산광역시,울주군,삼남면,129.108532992087,35.5409868486272


In [51]:
level1_list = list(df_level3['광역시도'].unique())
new_df3=[]

for Si_Do in level1_list:
  level2_list = list(df_level3[df_level3['광역시도']==Si_Do]['시군구'].unique())
  for SiGunGu in level2_list:
    level3_list = list(df_level3[(df_level3['광역시도']==Si_Do) & (df_level3['시군구']==SiGunGu)]['읍면동'].unique()) #시군구를 날씨데이터 포맷으로 바꾸기 전 level3_list를 만들어야한다!
    for Dong in level3_list:
      new_df3.append((Si_Do,SiGunGu,Dong))

print(len(new_df3))
new_df3[-20:]

913


[('인천광역시', '미추홀구', '주안동'),
 ('인천광역시', '미추홀구', '도화동'),
 ('인천광역시', '계양구', '동양동'),
 ('인천광역시', '계양구', '장기동'),
 ('울산광역시', '울주군', '언양읍'),
 ('울산광역시', '울주군', '삼남읍'),
 ('울산광역시', '울주군', '온산읍'),
 ('울산광역시', '울주군', '온양읍'),
 ('울산광역시', '울주군', '삼남면'),
 ('울산광역시', '북구', '화봉동'),
 ('울산광역시', '북구', '신천동'),
 ('울산광역시', '북구', '연암동'),
 ('울산광역시', '북구', '중산동'),
 ('울산광역시', '북구', '매곡동'),
 ('울산광역시', '북구', '천곡동'),
 ('울산광역시', '중구', '장현동'),
 ('세종특별자치시', '세종특별자치시', '소담동'),
 ('세종특별자치시', '세종특별자치시', '다정동'),
 ('세종특별자치시', '세종특별자치시', '고운동'),
 ('세종특별자치시', '세종특별자치시', '조치원읍')]

In [52]:
total_list

[('강원도', '속초시', '교동'),
 ('강원도', '속초시', '금호동'),
 ('강원도', '속초시', '조양동'),
 ('강원도', '원주시', '단구동'),
 ('강원도', '원주시', '일산동'),
 ('강원도', '원주시', '단계동'),
 ('강원도', '원주시', '무실동'),
 ('강원도', '원주시', '지정면'),
 ('강원도', '삼척시', '교동'),
 ('강원도', '삼척시', '남양동'),
 ('강원도', '강릉시', '송정동'),
 ('강원도', '철원군', '서면'),
 ('강원도', '철원군', '동송읍'),
 ('강원도', '춘천시', '동내면'),
 ('강원도', '춘천시', '석사동'),
 ('강원도', '춘천시', '퇴계동'),
 ('경기도', '고양시일산동구', '식사동'),
 ('경기도', '고양시일산동구', '정발산동'),
 ('경기도', '고양시일산동구', '중산동'),
 ('경기도', '고양시일산서구', '탄현동'),
 ('경기도', '고양시일산서구', '대화동'),
 ('경기도', '시흥시', '목감동'),
 ('경기도', '시흥시', '은행동'),
 ('경기도', '시흥시', '신천동'),
 ('경기도', '시흥시', '대야동'),
 ('경기도', '시흥시', '월곶동'),
 ('경기도', '시흥시', '장곡동'),
 ('경기도', '시흥시', '능곡동'),
 ('경기도', '시흥시', '매화동'),
 ('경기도', '의정부시', '가능동'),
 ('경기도', '의정부시', '녹양동'),
 ('경기도', '의정부시', '장암동'),
 ('경기도', '포천시', '소흘읍'),
 ('경기도', '포천시', '선단동'),
 ('경기도', '포천시', '신북면'),
 ('경기도', '포천시', '일동면'),
 ('경기도', '포천시', '군내면'),
 ('경기도', '김포시', '풍무동'),
 ('경기도', '김포시', '사우동'),
 ('경기도', '김포시', '고촌읍'),
 ('경기도', '김포시', '양촌

In [53]:
still_missing=[]
for region in new_df3:
  if region not in total_list:
    still_missing.append(region)

still_missing

[]

In [54]:
df_level3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 913 entries, 0 to 912
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   광역시도       913 non-null    object
 1   시군구        913 non-null    object
 2   읍면동        913 non-null    object
 3   경도(초/100)  913 non-null    object
 4   위도(초/100)  913 non-null    object
dtypes: object(5)
memory usage: 42.8+ KB


In [55]:
df_level3 = df_level3.astype({'경도(초/100)' : 'float',	'위도(초/100)': 'float'})
df_level3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 913 entries, 0 to 912
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   광역시도       913 non-null    object 
 1   시군구        913 non-null    object 
 2   읍면동        913 non-null    object 
 3   경도(초/100)  913 non-null    float64
 4   위도(초/100)  913 non-null    float64
dtypes: float64(2), object(3)
memory usage: 42.8+ KB


In [56]:
df_level3.describe()

,경도(초/100),위도(초/100)
count,913.000000,913.000000
mean,127.502530,36.475689
std,0.838608,1.170833
min,126.251812,33.226671
25%,126.850781,35.814011
50%,127.095595,36.768913
75%,128.207405,37.495771
max,129.386804,38.206370


In [57]:
df_level3.groupby('광역시도').describe()

경도(초/100)                        ...  위도(초/100)                      
            count        mean       std  ...        50%        75%        max
광역시도                                     ...                                 
강원도          26.0  128.131982  0.533931  ...  37.850454  38.093333  38.206370
경기도         227.0  126.989611  0.198357  ...  37.480046  37.691353  38.025317
경상남도         77.0  128.392950  0.340057  ...  35.197291  35.218241  35.537658
경상북도        100.0  128.777257  0.344275  ...  36.101288  36.563119  36.884933
광주광역시        23.0  126.888146  0.045108  ...  35.182506  35.205175  35.223565
대구광역시        42.0  128.563851  0.067803  ...  35.853842  35.874729  35.940122
대전광역시        22.0  127.420751  0.040666  ...  36.343853  36.352500  36.380222
부산광역시        21.0  129.044715  0.096902  ...  35.137894  35.187083  35.322375
서울특별시       125.0  126.970207  0.089599  ...  37.530436  37.562237  37.687396
세종특별자치시       4.0  127.271086  0.033476  ...  36.506877  36.540964  36.604528
울산광역시        12.0  129.286871  0.107951  ...  35.585742  35.638599  35.663797
인천광역시        17.0  126.706187  0.034635  ...  37.493978  37.517603  37.612151
전라남도         34.0  127.233945  0.475682  ...  34.959947  35.030384  35.048144
전라북도         63.0  126.773969  0.126446  ...  35.972236  35.983523  35.999661
제주특별자치도      43.0  126.525260  0.132641  ...  33.292955  33.483876  33.531178
충청남도         35.0  126.845671  0.301886  ...  36.771463  36.796171  36.936764
충청북도         42.0  128.041494  0.242211  ...  37.123286  37.136976  37.194646

[17 rows x 16 columns]

In [58]:
# df_level3.to_csv("/content/drive/MyDrive/경도_위도_level3.csv", index=False, encoding="utf-8-sig")